In [13]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from IPython.display import Image

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

In [14]:
df = pd.read_csv('data/test.csv')
df.head()

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15
0,425552,2020-01-23 01:26:00,2020-01-23 02:52:00,2020-01-22 20:26:00,2020-01-22 21:52:00,LGA,12953,BGR,10581,N8894A,9E,20363,5032,378.0,0,NaN
1,415418,2020-01-23 01:26:00,2020-01-23 03:04:00,2020-01-22 19:26:00,2020-01-22 22:04:00,MDW,13232,PHL,14100,N8615E,WN,19393,1168,668.0,0,NaN
2,414222,2020-01-23 01:26:00,2020-01-23 03:06:00,2020-01-22 18:26:00,2020-01-22 19:06:00,DEN,11292,LAS,12889,N7853B,WN,19393,799,628.0,0,NaN
3,421432,2020-01-23 01:26:00,2020-01-23 02:48:00,2020-01-22 17:26:00,2020-01-22 18:48:00,SJC,14831,SAN,14679,N7812G,WN,19393,4454,417.0,0,0.0
4,421356,2020-01-23 01:26:00,2020-01-23 02:24:00,2020-01-22 17:26:00,2020-01-22 18:24:00,SJC,14831,BUR,10800,N7859B,WN,19393,2024,296.0,0,NaN


In [51]:
df[df.DISTANCE==0].count()

ID                       0
DATE_DEPARTURE_UTC       0
DATE_ARRIVAL_UTC         0
DATE_DEPARTURE_LCL       0
DATE_ARRIVAL_LCL         0
ORIGIN                   0
ORIGIN_AIRPORT_ID        0
DEST                     0
DEST_AIRPORT_ID          0
TAIL_NUM                 0
OP_CARRIER               0
OP_CARRIER_AIRLINE_ID    0
OP_CARRIER_FL_NUM        0
DISTANCE                 0
CANCELLED                0
DEP_DEL15                0
dtype: int64

In [15]:
df2 = pd.read_csv('data/train.csv')
df2.head()

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
0,345355,2020-01-18 22:10:00,2020-01-18 23:59:00,2020-01-18 15:10:00,2020-01-18 15:59:00,SLC,14869,SNA,14908,N128DU,DL,19790,1056,588.0,0,0.0,0
1,323447,2020-01-17 21:47:00,2020-01-18 04:10:00,2020-01-17 16:47:00,2020-01-17 20:10:00,BOS,10721,SEA,14747,N486AS,AS,19930,15,2496.0,0,NaN,0
2,242237,2020-01-13 13:26:00,2020-01-13 14:26:00,2020-01-13 07:26:00,2020-01-13 08:26:00,DAL,11259,HOU,12191,N901WN,WN,19393,3,239.0,0,NaN,0
3,271408,2020-01-15 02:45:00,2020-01-15 03:37:00,2020-01-14 20:45:00,2020-01-14 21:37:00,STL,15016,DSM,11423,N778SW,WN,19393,1588,259.0,0,0.0,0
4,282447,2020-01-15 23:22:00,2020-01-16 00:49:00,2020-01-15 18:22:00,2020-01-15 19:49:00,CLT,11057,CHS,10994,N579NN,OH,20397,5207,168.0,0,NaN,1


In [8]:
df2.shape

(419487, 17)

In [11]:
data = pd.read_csv("data/train.csv") 
for i in data.columns:
    print(i)
    print("unique values:", data[i].unique())
    print("Nr unique values:", len(data[i].unique()))
    print("value_counts:", data[i].value_counts())

ID
unique values: [345355 323447 242237 ... 365119  71648 224261]
Nr unique values: 419487
value_counts: 2047      1
115858    1
70792     1
72841     1
66698     1
         ..
270801    1
276946    1
274899    1
264660    1
0         1
Name: ID, Length: 419487, dtype: int64
DATE_DEPARTURE_UTC
unique values: ['2020-01-18 22:10:00' '2020-01-17 21:47:00' '2020-01-13 13:26:00' ...
 '2020-01-15 05:32:00' '2020-01-10 07:58:00' '2020-01-17 08:49:00']
Nr unique values: 29661
value_counts: 2020-01-09 11:55:00    52
2020-01-06 11:56:00    49
2020-01-17 11:55:00    48
2020-01-16 11:54:00    47
2020-01-16 12:55:00    46
                       ..
2020-01-12 06:02:00     1
2020-01-22 07:56:00     1
2020-01-10 06:09:00     1
2020-01-18 07:48:00     1
2020-01-18 10:15:00     1
Name: DATE_DEPARTURE_UTC, Length: 29661, dtype: int64
DATE_ARRIVAL_UTC
unique values: ['2020-01-18 23:59:00' '2020-01-18 04:10:00' '2020-01-13 14:26:00' ...
 '2020-01-23 05:44:00' '2020-01-17 07:45:00' '2020-01-22 07:55:00']
Nr

value_counts: ATL    22393
ORD    17193
DFW    16746
DEN    14202
CLT    13959
       ...  
BFM        9
ADK        7
PPG        6
OGD        6
OWB        6
Name: DEST, Length: 350, dtype: int64
DEST_AIRPORT_ID
unique values: [14908 14747 12191 11423 10994 10693 11298 14107 11433 10397 13198 12266
 10821 13303 11259 12953 10849 11618 11697 12889 14570 15304 14783 14492
 12264 13487 14576 15376 14635 12478 12892 13232 14869 14771 11292 11057
 11638 15016 14831 11193 13204 13244 14057 13930 11278 12992 10721 14193
 13830 10792 13476 14100 14027 12173 13871 13931 11003 13495 11995 12951
 13891 11823 12954 10423 11042 15024 14679 13796 10155 12758 15370 11986
 12323 10529 12945 12197 14262 10299 14843 14730 10158 14689 11996 12982
 12448 12339 15041 12343 13795 11503 14512 13342 14893 12451 12278 10713
 13486 14683 13851 16218 12896 10431 14122 10785 13290 11540 14794 12217
 14986 11267 12244 11537 11973 12402 11066 12206 15412 14524 10257 10800
 10781 11884 13256 10561 13360 11140 15356 1

In [18]:
df2.isnull().head()

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


In [21]:
df2.DEP_DEL15.isnull().value_counts()

False    272667
True     146820
Name: DEP_DEL15, dtype: int64

In [24]:
duplicated_mask = df2.duplicated(keep='first')
print('Number of duplicates:', duplicated_mask.sum())

Number of duplicates: 0


In [26]:
#target variable
df2.ARR_DEL15.isnull().sum()

0

In [28]:
col_df2_target = df2.ARR_DEL15
col_df2_target

0         0
1         0
2         0
3         0
4         1
         ..
419482    0
419483    0
419484    0
419485    0
419486    1
Name: ARR_DEL15, Length: 419487, dtype: int64

In [30]:
df2.tail()

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
419482,374775,2020-01-21 00:54:00,2020-01-21 01:58:00,2020-01-20 16:54:00,2020-01-20 17:58:00,ONT,13891,OAK,13796,N278WN,WN,19393,1475,362.0,0,0.0,0
419483,295665,2020-01-16 17:28:00,2020-01-16 20:18:00,2020-01-16 12:28:00,2020-01-16 15:18:00,FLL,11697,JFK,12478,N2017J,B6,20409,1202,1069.0,0,0.0,0
419484,365119,2020-01-20 00:37:00,2020-01-20 03:51:00,2020-01-19 18:37:00,2020-01-19 19:51:00,IAH,12266,LAS,12889,N832UA,UA,19977,1044,1222.0,0,0.0,0
419485,71648,2020-01-04 21:43:00,2020-01-05 00:19:00,2020-01-04 15:43:00,2020-01-04 19:19:00,MKE,13342,MCO,13204,N286WN,WN,19393,3487,1066.0,0,NaN,0
419486,224261,2020-01-12 22:21:00,2020-01-12 23:05:00,2020-01-12 15:21:00,2020-01-12 16:05:00,PHX,14107,TUS,15376,N907FJ,YV,20378,5756,NaN,0,NaN,1


In [31]:
df2[df2.ARR_DEL15==1]

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
4,282447,2020-01-15 23:22:00,2020-01-16 00:49:00,2020-01-15 18:22:00,2020-01-15 19:49:00,CLT,11057,CHS,10994,N579NN,OH,20397,5207,168.0,0,NaN,1
12,186898,2020-01-11 00:52:00,2020-01-11 03:34:00,2020-01-10 19:52:00,2020-01-10 21:34:00,ATL,10397,MCI,13198,N836DN,DL,19790,1217,692.0,0,0.0,1
13,323323,2020-01-17 11:31:00,2020-01-17 16:58:00,2020-01-17 06:31:00,2020-01-17 10:58:00,BOS,10721,IAH,12266,N827JB,B6,20409,1931,1597.0,0,NaN,1
20,213353,2020-01-12 01:41:00,2020-01-12 03:48:00,2020-01-11 20:41:00,2020-01-11 22:48:00,ATL,10397,FLL,11697,N697DL,DL,19790,346,581.0,0,0.0,1
24,380003,2020-01-20 21:35:00,2020-01-20 23:43:00,2020-01-20 14:35:00,2020-01-20 15:43:00,PHX,14107,RNO,14570,N808AW,AA,19805,1888,601.0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419460,367965,2020-01-20 05:36:00,2020-01-20 08:16:00,2020-01-19 21:36:00,2020-01-20 00:16:00,SEA,14747,LAX,12892,N842VA,AS,19930,1041,954.0,0,1.0,1
419462,284812,2020-01-16 01:23:00,2020-01-16 02:16:00,2020-01-15 15:23:00,2020-01-15 16:16:00,HNL,12173,KOA,12758,N494HA,HA,19690,348,163.0,0,NaN,1
419472,353995,2020-01-19 15:33:00,2020-01-19 17:34:00,2020-01-19 10:33:00,2020-01-19 12:34:00,ATL,10397,ROC,14576,N913DN,DL,19790,2146,749.0,0,NaN,1
419475,85043,2020-01-05 13:04:00,2020-01-05 19:15:00,2020-01-05 08:04:00,2020-01-05 11:15:00,EWR,11618,LAS,12889,N656NK,NK,20416,993,2227.0,0,NaN,1


In [33]:
df2[df2.ARR_DEL15==1].shape

(62195, 17)

In [ ]:
df2_drop = df2.drop([ID	DATE_DEPARTURE_UTC	DATE_ARRIVAL_UTC	DATE_DEPARTURE_LCL	DATE_ARRIVAL_LCL	ORIGIN	ORIGIN_AIRPORT_ID	DEST	DEST_AIRPORT_ID	TAIL_NUM	OP_CARRIER	OP_CARRIER_AIRLINE_ID	OP_CARRIER_FL_NUM	DISTANCE	CANCELLED	DEP_DEL15	ARR_DEL15])

In [35]:
df2.TAIL_NUM.duplicated(keep='first').sum()

414111

In [36]:
df2.OP_CARRIER_FL_NUM.duplicated(keep='first').sum()

412772

In [37]:
df2.head()

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
0,345355,2020-01-18 22:10:00,2020-01-18 23:59:00,2020-01-18 15:10:00,2020-01-18 15:59:00,SLC,14869,SNA,14908,N128DU,DL,19790,1056,588.0,0,0.0,0
1,323447,2020-01-17 21:47:00,2020-01-18 04:10:00,2020-01-17 16:47:00,2020-01-17 20:10:00,BOS,10721,SEA,14747,N486AS,AS,19930,15,2496.0,0,NaN,0
2,242237,2020-01-13 13:26:00,2020-01-13 14:26:00,2020-01-13 07:26:00,2020-01-13 08:26:00,DAL,11259,HOU,12191,N901WN,WN,19393,3,239.0,0,NaN,0
3,271408,2020-01-15 02:45:00,2020-01-15 03:37:00,2020-01-14 20:45:00,2020-01-14 21:37:00,STL,15016,DSM,11423,N778SW,WN,19393,1588,259.0,0,0.0,0
4,282447,2020-01-15 23:22:00,2020-01-16 00:49:00,2020-01-15 18:22:00,2020-01-15 19:49:00,CLT,11057,CHS,10994,N579NN,OH,20397,5207,168.0,0,NaN,1


In [39]:
df2[df2.OP_CARRIER_FL_NUM==3]

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
2,242237,2020-01-13 13:26:00,2020-01-13 14:26:00,2020-01-13 07:26:00,2020-01-13 08:26:00,DAL,11259,HOU,12191,N901WN,WN,19393,3,239.0,0,NaN,0
9339,134158,2020-01-07 21:17:00,2020-01-08 01:09:00,2020-01-07 16:17:00,2020-01-07 21:09:00,JFK,12478,SJU,14843,N2002J,B6,20409,3,1598.0,0,1.0,1
9634,182767,2020-01-10 20:05:00,2020-01-10 23:42:00,2020-01-10 15:05:00,2020-01-10 19:42:00,JFK,12478,SJU,14843,N948JB,B6,20409,3,1598.0,0,0.0,0
13958,88650,2020-01-05 18:15:00,2020-01-05 21:32:00,2020-01-05 10:15:00,2020-01-05 15:32:00,OAK,13796,STL,15016,N7835A,WN,19393,3,1725.0,0,NaN,0
15893,362921,2020-01-19 13:00:00,2020-01-19 15:50:00,2020-01-19 07:00:00,2020-01-19 10:50:00,ORD,13930,FLL,11697,N614NK,NK,20416,3,1182.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410511,311831,2020-01-16 20:24:00,2020-01-17 00:08:00,2020-01-16 15:24:00,2020-01-16 20:08:00,JFK,12478,SJU,14843,N954JB,B6,20409,3,1598.0,0,1.0,1
411489,266176,2020-01-14 12:55:00,2020-01-14 16:04:00,2020-01-14 06:55:00,2020-01-14 11:04:00,ORD,13930,FLL,11697,N669NK,NK,20416,3,NaN,0,0.0,0
412361,61507,2020-01-04 22:17:00,2020-01-05 04:28:00,2020-01-04 17:17:00,2020-01-04 20:28:00,DCA,11278,SEA,14747,N926VA,AS,19930,3,2329.0,0,0.0,0
414936,285074,2020-01-15 17:25:00,2020-01-15 23:51:00,2020-01-15 12:25:00,2020-01-15 15:51:00,JFK,12478,LAX,12892,N107NN,AA,19805,3,2475.0,0,0.0,0


In [46]:
df2_DEP_DEL15_nan = df2[df2.DEP_DEL15==np.nan]
df2_DEP_DEL15_nan

,ID,DATE_DEPARTURE_UTC,DATE_ARRIVAL_UTC,DATE_DEPARTURE_LCL,DATE_ARRIVAL_LCL,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,TAIL_NUM,OP_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15


In [47]:
df_reduce = df2[['ORIGIN','ORIGIN_AIRPORT_ID','DEST','DEST_AIRPORT_ID','OP_CARRIER_AIRLINE_ID','OP_CARRIER_FL_NUM','DISTANCE','CANCELLED','DEP_DEL15','ARR_DEL15']]

In [64]:
df_reduce

,ORIGIN,ORIGIN_AIRPORT_ID,DEST,DEST_AIRPORT_ID,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,DISTANCE,CANCELLED,DEP_DEL15,ARR_DEL15
0,SLC,14869,SNA,14908,19790,1056,588.0,0,0.0,0
1,BOS,10721,SEA,14747,19930,15,2496.0,0,NaN,0
2,DAL,11259,HOU,12191,19393,3,239.0,0,NaN,0
3,STL,15016,DSM,11423,19393,1588,259.0,0,0.0,0
4,CLT,11057,CHS,10994,20397,5207,168.0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
419482,ONT,13891,OAK,13796,19393,1475,362.0,0,0.0,0
419483,FLL,11697,JFK,12478,20409,1202,1069.0,0,0.0,0
419484,IAH,12266,LAS,12889,19977,1044,1222.0,0,0.0,0
419485,MKE,13342,MCO,13204,19393,3487,1066.0,0,NaN,0


In [55]:
df_reduce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419487 entries, 0 to 419486
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ORIGIN                 419487 non-null  object 
 1   ORIGIN_AIRPORT_ID      419487 non-null  int64  
 2   DEST                   419487 non-null  object 
 3   DEST_AIRPORT_ID        419487 non-null  int64  
 4   OP_CARRIER_AIRLINE_ID  419487 non-null  int64  
 5   OP_CARRIER_FL_NUM      419487 non-null  int64  
 6   DISTANCE               401869 non-null  float64
 7   CANCELLED              419487 non-null  int64  
 8   DEP_DEL15              272667 non-null  float64
 9   ARR_DEL15              419487 non-null  int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 32.0+ MB


In [66]:
 s1 = df_reduce.DEP_DEL15.fillna(0)


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
419482    0.0
419483    0.0
419484    0.0
419485    0.0
419486    0.0
Name: DEP_DEL15, Length: 419487, dtype: float64

# how to fill nan values with the mode() of corresponding ...

In [84]:
df_reduce.DEP_DEL15.fillna('False')

0             0
1         False
2         False
3             0
4         False
          ...  
419482        0
419483        0
419484        0
419485    False
419486    False
Name: DEP_DEL15, Length: 419487, dtype: object

In [88]:
df_reduce.columns

Index(['ORIGIN', 'ORIGIN_AIRPORT_ID', 'DEST', 'DEST_AIRPORT_ID',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'DISTANCE', 'CANCELLED',
       'DEP_DEL15', 'ARR_DEL15'],
      dtype='object')

In [89]:
df_big_reduce = df_reduce.drop(['ORIGIN', 'ORIGIN_AIRPORT_ID', 'DEST', 'DEST_AIRPORT_ID',
       'OP_CARRIER_AIRLINE_ID','DISTANCE', 'CANCELLED',
       ], axis=1)

In [90]:
df_big_reduce 

,OP_CARRIER_FL_NUM,DEP_DEL15,ARR_DEL15
0,1056,0,0
1,15,False,0
2,3,False,0
3,1588,0,0
4,5207,False,1
...,...,...,...
419482,1475,0,0
419483,1202,0,0
419484,1044,0,0
419485,3487,False,0


In [94]:
df_big_reduce.DEP_DEL15.values

array([0.0, 'False', 'False', ..., 0.0, 'False', 'False'], dtype=object)

In [96]:
df_big_reduce[df_big_reduce.DEP_DEL15==True]
        

,OP_CARRIER_FL_NUM,DEP_DEL15,ARR_DEL15
17,5956,1,0
32,1900,1,1
40,4513,1,0
56,4451,1,0
67,2093,1,0
...,...,...,...
419422,671,1,0
419425,3751,1,0
419456,752,1,0
419458,5352,1,0
